# resp_evoked (quantifying response to evoked activity in a given session)
IMPORTANT: Keep this notebook as identical to `resp_photostim.ipynb` as possible.

Both scripts do 'trial-based' analysis, where each trial has some start/end time and identity.

The identity can be:
    1) evoked stimulation trial type (currently they are all the same)
    2) photostim stimulation trial type / the identity of the mark point being stimulated

The outputs for the 'resp_map-related' part are:
    1) 'response map'(s) of the movie field of view for each evoked trial (+summary over trials) - response of network to a particular trial type of evoked activity
    2) 'response map'(s) of the movie field of view for each photostim trial (+summary over trials) - response of network to a particular single cell stimulation

The outputs for the 'suite2p-related' part are:
    1) which cells are responding to a particular evoked stimulation
    2) which cells are responding to a particular photostim stimulation (the stimulated cell as well as all other cells)


In [ ]:
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
# get five random numbers between 0 and 1312
np.random.seed(123)
rand_nrs = np.random.randint(0, 100, size=15)
print(rand_nrs)

In [ ]:
data_dir = 'data_proc' # data_loc is  the directory on local ssd (only two sessions, one for jm049 and one for jm048)
experimenter = 'jm'
mouse = 'jm064' # 'jm049' or 'jm048'
t2p_s2p_dir = 'track2p/matched_suite2p'

stim_wind = (10, 120)

# plot_nrns = [7, 15, 226, 116, 67, 0, 1, 2, 3, 4, 684, 559, 1216, 835, 763] # jm055
# plot_nrns = [0, 1, 14, 35, 51, 4, 8, 22, 29, 2, 684, 559, 629, 192, 835] # jm056
# get 15 random numbers between 0 and 500
plot_nrns = rand_nrs.tolist()
# labels = ['resp', 'resp', 'resp', 'resp', 'resp', 'state', 'state', 'state', 'state', 'state', 'rand', 'rand', 'rand', 'rand', 'rand']
labels = ['rand'] * 15

In [ ]:
# channel: 2
# plane: 0
# frame_period: 0.033602476  # metadata-derived frame period for 30Hz acquisition
# fov_shape: [512, 512]      # shape of the FOV in pixels

# # baseline and response parameters
# bsln_n_frames: 10          # baseline window in frames
# resp_n_frames: 10          # response window in frames

# bsln_sub_type: "trial_by_trial"  # 'trial_by_trial' or 'session_wide'

# # spatial extent of response
# n_dist_bins: 724           # number of distance bins (724 = diagonal of 512x512)

# # visualization parameters
# n_rows_fov: 4
# vlim: 200
# txt_shift: [7, 7]
# sat_perc_fov: 99.99
# peristim_wind: [10, 30]
# zoomin_npix: 128

# dist_bins_xlim: 362        # 724 // 2
# dist_bins_xlim_zoom: 45    # 724 // 16

In [ ]:
import os

In [ ]:
# for now go to the fake suitep2p directory and load all the Fs from theree
matched_s2p_path = os.path.join(data_dir, experimenter, mouse, t2p_s2p_dir)
# now get all the subdirectories and load F
all_matched_s2p_paths = [os.path.join(matched_s2p_path, d) for d in os.listdir(matched_s2p_path) if os.path.isdir(os.path.join(matched_s2p_path, d))]
all_matched_s2p_paths.sort()  # sort to have a consistent order

all_F = []
all_stim_protocol = []
all_stim_times = []

for path in all_matched_s2p_paths:

    print(f'Loading from {path}')
    F = np.load(os.path.join(path, 'suite2p', 'plane0', 'F.npy'))  # shape (n_neurons, n_frames)

    session = os.path.basename(path)
    stim_protocol = np.load(f'{data_dir}/{experimenter}/{mouse}/{session}/stim_protocol.npy')
    stim_times = np.load(f'{data_dir}/{experimenter}/{mouse}/{session}/stim_times.npy')

    all_F.append(F)
    all_stim_protocol.append(stim_protocol)
    all_stim_times.append(stim_times)


In [ ]:
all_stim_times

In [ ]:
for (i,F) in enumerate(all_F):
    for j in plot_nrns:
        plt.figure(figsize=(20, 2))
        plt.plot(F[j], c=f'C{i}')
        for st in stim_times:
            plt.axvline(st, color='r')

In [ ]:
stim_array = np.zeros((len(all_F), len(plot_nrns), len(stim_times), stim_wind[0] + stim_wind[1]))

for k in range(len(all_F)):
    for i in range(len(plot_nrns)):
        for j, st in enumerate(all_stim_times[k]):
            stim_array[k, i, j, :] = all_F[k][plot_nrns[i], st - stim_wind[0]:st + stim_wind[1]]
            # subtract baseline
            stim_array[k, i, j, :] = stim_array[k, i, j, :] - np.mean(stim_array[k, i, j, :stim_wind[0]])
        

In [ ]:
# now plot the heatmap for each neuron
for k in range(len(all_F)):
    this_stim_array = stim_array[k, :, :, :]  # average over stim times
    fig, axs = plt.subplots(3, 5, figsize=(10, 5), dpi=300)
    for i in range(len(plot_nrns)):
        ax = axs[i // 5, i % 5]
        plt.sca(ax)
        plt.imshow(this_stim_array[i, :, :], aspect='auto', cmap='bwr', vmin = -5*np.std(this_stim_array[i, :, :]), vmax = 5*np.std(this_stim_array[i, :, :]))
        # if first column, add ylabel
        # remove xticks and yticks
        plt.xticks([])
        plt.yticks([])
        if i % 5 == 0:
            plt.ylabel(labels[i])
            # add yticks
            plt.yticks([0, this_stim_array.shape[1]//2, this_stim_array.shape[1]-1], [0, this_stim_array.shape[1]//2, this_stim_array.shape[1]-1])
        # if last row, add xlabel
        if i // 5 == 2:
            plt.xlabel('Time (frames)')
            plt.xticks([stim_wind[0], stim_wind[0]+stim_wind[1]//2, stim_wind[0]+stim_wind[1]], [0, stim_wind[1]//2, stim_wind[1]])
                


In [ ]:
for k in range(len(all_F)):
    this_stim_array = stim_array[k, :, :, :]  # average over stim times
    # plot the average +-std for resp neurons
    fig, axs = plt.subplots(1, 5, figsize=(10, 1.5), dpi=300)

    for i in range(5):
        ax = axs[i]
        plt.sca(ax)
        mean_trace = np.mean(this_stim_array[i, :, :], axis=0)
        std_trace = np.std(this_stim_array[i, :, :], axis=0)
        plt.plot(mean_trace, color=f'C{i}')
        plt.fill_between(np.arange(len(mean_trace)), mean_trace - std_trace, mean_trace + std_trace, color=f'C{i}', alpha=0.1)
        plt.title(f'Resp neuron {plot_nrns[i]}')
        plt.axvline(stim_wind[0], color='grey', linestyle='--')
        plt.xlabel('Time (frames)')
        if i == 0:
            plt.ylabel('dF/F')
        plt.xticks([stim_wind[0], stim_wind[0]+stim_wind[1]//2, stim_wind[0]+stim_wind[1]], [0, stim_wind[1]//2, stim_wind[1]])
        # keep y ticks but remove labels
        ax.set_yticklabels([])
        # remove top and right spines
        ax.spines['top'].set_visible(False)
        ax.spines['right'].set_visible(False)


In [ ]:
# now show all neurons together
fig, axs = plt.subplots(3, 5, figsize=(10, 5), dpi=300)
for i in range(len(plot_nrns)):
    ax = axs[i // 5, i % 5]
    plt.sca(ax)
    mean_trace = np.mean(stim_array[i, :, :], axis=0)
    std_trace = np.std(stim_array[i, :, :], axis=0)
    plt.plot(mean_trace, color=f'C{i}')
    plt.fill_between(np.arange(len(mean_trace)), mean_trace - std_trace, mean_trace + std_trace, color=f'C{i}', alpha=0.1)
    plt.axvline(stim_wind[0], color='grey', linestyle='--')
    if i % 5 == 0:
        plt.ylabel('dF/F')
    plt.xticks([stim_wind[0], stim_wind[0]+stim_wind[1]//2, stim_wind[0]+stim_wind[1]], [0, stim_wind[1]//2, stim_wind[1]])

    if i // 5 == 2:
        plt.xlabel('Time (frames)')
    # keep y ticks but remove labels
    ax.set_yticklabels([])
    # remove top and right spines
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)   